# Convert pt(npy) to midi

In [1]:
import numpy as np
import torch
import pretty_midi
import pandas as pd
import collections
import glob
import os
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [ ]:
def convert_to_midi(vector, note_split):
    """
    convert to midi
    
    params
    -------------
    vector: numpy.ndarray
        vector(channel, length, 128)
    note_split: int
        time step of note
    
    returns
    ---------
    piano: object
        midi_instrument
    """
    vector = ((vector +1)*63.5).clip(0, 127)
    #vector[vector >0.5] = 70
    shapes = vector.shape
    instrument_name = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=instrument_name)

    #params
    start_time = 0.0
    end_time = 0.0
    tempo = 120
    time_split = 60/(tempo*note_split/4)

    #add notes
    for p in range(shapes[-1]):
        velocity = 0
        start_time = 0.0
        end_time = 0.0
        durting = False #extend note
        on = 0 #ringing note
        for t in range(shapes[-2]):
            if not durting:
                on = int(round(vector[1][t][p]))
                velocity = int(round(vector[0][t][p]))
            end_time += time_split
    
            #process of next note
            if on > 0:
                if t != shapes[-2]-1:
                    sutain = int(round(vector[0][t+1][p]))
                    if sutain > 0 and int(round(vector[1][t+1][p])) < 1:
                        durting = True
                        continue
                    else:
                        durting = False
            else:
                start_time = end_time
                durting = False
                continue      
    
            #add note
            note = pretty_midi.Note(
                velocity=velocity,
                pitch=int(p),
                start=start_time, 
                end=end_time
            )
            piano.notes.append(note)
            start_time = end_time
    
    return piano

In [ ]:
#file names
file_name = "test/sample.pt"
data_v = "test"
expect = False
#params
data_num = 40 
note_split = 128 #time step of note
observation_names = ['midi'] #observation names
result_dir = f"result_midi/{file_name}" 

#load file
os.makedirs(result_dir, exist_ok=True)
vectors_test = [[]for i in range(len(observation_names))]
vectors_recon = [[]for i in range(len(observation_names))]
vectors_imag = [[]for i in range(len(observation_names))]

vector_result = torch.load(f"result/{file_name}", map_location='cpu')
if type(vector_result) == dict:
    for i in range(data_num):
        for n, key in enumerate(observation_names):
            vector_tmp = vector_result[key][i].detach().cpu().numpy()
            vectors_imag[n].append(vector_tmp)
else:
    for i in range(data_num):
        for n, key in enumerate(observation_names):
            vector_tmp = vector_result[i].detach().cpu().numpy()
            vectors_imag[n].append(vector_tmp)
        
#convert to midi
for i in tqdm(range(data_num)):
    midi = pretty_midi.PrettyMIDI()
    instrument = []
    for n in range(len(observation_names)):
        instrument.append(convert_to_midi(vectors_imag[n][i], note_split))
    for n in range(len(observation_names)):
        midi.instruments.append(instrument[n])
    
    #write midi
    wirte_title = os.path.join(result_dir, str(i) + ".mid")
    midi.write(wirte_title)

100%|██████████| 40/40 [00:21<00:00,  1.88it/s]
